# NER using GPT-3.5

### Project name: Honos
Date: 24th May 2024

Author: Milindi Kodikara

Supervisor: Professor Karin Verspoor

Guidelines before running the notebook:
1. Add the correct filename paths for train data, test data, dev data

Libraries used: 
- pd
- numpy

In [ ]:
import pandas as pd
import json


### Step 1: Load and pre-process data and prompt library 


#### Step 1.1: Load datasets

In [ ]:
# train_text.tsv
# pmid\tfilename\ttext

train_data = pd.read_csv("./genovardis_train_dev/train_text.tsv", sep='\t', header=0)

train_data.sample(5)

In [ ]:
# train_annotations.tsv
# pmid\tfilename\tmark\tlabel\toffset1\toffset2\tspan

train_annotations = pd.read_csv("./genovardis_train_dev/train_annotation.tsv", sep='\t', header=0)

train_annotations.sample(5)


#### Step 1.2: Load prompt library

In [ ]:

prompt_library = pd.read_json('prompts.json')

prompt_library


#### Step 1.3: Create data+prompt dataset


### Step 2: Setting up GPT-3.5

### Step 3: Post-processing


### Step 4: Evaluation

*Skip this part for evaluation dataset as there is no gold standard data to compare against.*


### Step 5: Saving output

`.tsv` file containing the annotations in the following format: 

`pmid   filename   label   offset1   offset2   span`.

